# ReAct Pattern Demonstration

This Jupyter notebook demonstrates a **ReAct (Reason + Act) pattern** applied to a simple airline refund scenario. We break the logic into **four sections**:
1. **Policy Logic**
2. **LLM Adapter**
3. **ReAct Agent**
4. **Main Orchestration**

We’ll simulate an airline chatbot that decides whether a passenger is eligible for a refund. The ReAct approach is:
- **Reason**: Form an internal chain of thought.
- **Act**: Possibly call an external "tool" (in this case, the policy logic) to get data.
- **Observe**: Incorporate tool results.
- **Conclude**: Return a final user-facing answer.

Let's walk through each section in detail.

## Section 1: Policy Logic
In this section, we define the airline’s **refund policy rules** and a helper function that checks refund eligibility. In reality, these policies might live in a database or be more complex, but here we store them in a simple dictionary. The function returns whether the passenger is eligible for a refund and, if so, the refund ratio.

In [ ]:
from datetime import datetime, timedelta

# Example airline policies stored as data
AIRLINE_POLICIES = {
    "REFUND_WINDOW_HOURS": 24,  # full refund if canceled within 24 hours of booking
    "NON_REFUNDABLE_CLASSES": ["BasicEconomy"],
    "PARTIAL_REFUND_CLASSES": ["Economy", "PremiumEconomy"],
    "FULL_REFUND_CLASSES": ["Business", "First"]
}

def check_refund_eligibility(booking_time, cabin_class):
    """
    Check if the flight is eligible for a refund based on booking time and cabin class.
    Returns a tuple: (eligible: bool, refund_amount: float)
    """
    now = datetime.now()
    time_since_booking = (now - booking_time).total_seconds() / 3600.0  # hours

    # 1. Check if it's within the full-refund window (e.g., 24 hours of booking)
    if time_since_booking <= AIRLINE_POLICIES["REFUND_WINDOW_HOURS"]:
        return True, 1.0  # Full refund

    # 2. Check cabin class rules
    if cabin_class in AIRLINE_POLICIES["NON_REFUNDABLE_CLASSES"]:
        return False, 0.0
    elif cabin_class in AIRLINE_POLICIES["FULL_REFUND_CLASSES"]:
        return True, 1.0
    elif cabin_class in AIRLINE_POLICIES["PARTIAL_REFUND_CLASSES"]:
        return True, 0.5  # Partial refund (50%)

    # Default case: no refund
    return False, 0.0


## Section 2: LLM Adapter
Here, we define a **placeholder function** that simulates a Large Language Model (LLM) call. In a production setting, this function could be replaced with real API calls (e.g., OpenAI, Anthropic, etc.). For demonstration, it returns simple mock responses when certain strings (like `Action: CallPolicyTool`) are detected.

In [ ]:
def call_llm(prompt, temperature=0.2):
    """
    Simulated LLM call. In production, you'd integrate with an actual LLM provider.
    Returns text that the LLM might produce given the prompt.
    """
    # Mock logic: we check if the prompt mentions "policy_tool" and respond accordingly
    if "Action: CallPolicyTool" in prompt:
        # The agent is asking for policy info
        return "Observation: The policy tool indicates partial refund eligibility.\n"
    elif "Action: SummarizeRefund" in prompt:
        return "Observation: The passenger is eligible for a partial refund of 50%.\n"
    else:
        # A default fallback for final reasoning
        return "This is my final answer based on the prior observations."


## Section 3: ReAct Agent
In this section, we **implement the ReAct pattern**:
1. The agent "reasons" (generates an internal thought process).
2. The agent "acts" by calling an external tool (in this case, our **Policy Logic** function) if needed.
3. The agent "observes" the result.
4. The agent either repeats more reasoning or concludes.

Our example agent:
- Calls the LLM to generate a step-by-step reasoning plan (mocked here).
- Calls `check_refund_eligibility(...)` to see if the passenger qualifies.
- Logs an observation.
- Asks the LLM to produce a final summary.
- Returns a user-friendly answer.

In [ ]:
class ReActAirlineAgent:
    def __init__(self):
        # Store any conversation context or memory if needed
        self.conversation_log = []
    
    def handle_refund_request(self, booking_time, cabin_class):
        """
        Handle a refund request using ReAct logic:
        1. Reason about the user's question
        2. Potentially call an external 'tool' (our policy function)
        3. Observe results
        4. Provide final answer
        """
        # STEP 1: The agent starts reasoning
        reason_prompt = (
            "The user wants a refund. Let's reason step by step.\n"
            "Step 1: We need to check the airline policy.\n"
            "Action: CallPolicyTool\n"
        )
        thought_response = call_llm(reason_prompt)
        
        # STEP 2: Actually call the policy check function for real data
        eligible, refund_ratio = check_refund_eligibility(booking_time, cabin_class)

        # Log the real observation
        observation_text = (
            f"Observation: Real policy check => eligible={eligible}, ratio={refund_ratio}\n"
        )
        self.conversation_log.append(observation_text)

        # STEP 3: Feed the observation back to the LLM for further reasoning
        second_prompt = (
            f"Given the observation that the passenger is eligible={eligible} with ratio={refund_ratio},\n"
            f"decide the final outcome.\n"
            "Action: SummarizeRefund\n"
        )
        second_response = call_llm(second_prompt)

        # STEP 4: Formulate the final user-facing decision
        if eligible:
            final_refund_amount = f"{int(refund_ratio * 100)}% of the ticket price."
            final_decision = f"Approved for a refund of {final_refund_amount}"
        else:
            final_decision = "Not eligible for a refund."

        final_prompt = (
            "Now produce a short user-facing answer that states the refund decision.\n"
        )
        final_response = call_llm(final_prompt)

        # STEP 5: Return a formatted message
        user_facing_answer = (
            f"Thank you for contacting Airline Support.\n"
            f"After reviewing your booking, you are: {final_decision}\n"
        )

        return user_facing_answer


## Section 4: Main Orchestration
Finally, we bring everything together. We simulate a user requesting a refund by providing a **booking time** and a **cabin class**, then let our ReAct-based agent determine the eligibility and refund amount.

In [ ]:
# Example usage
if __name__ == "__main__":
    # 1. Simulate a user booking time (e.g., booked 30 hours ago)
    simulated_booking_time = datetime.now() - timedelta(hours=30)
    
    # 2. Simulate a cabin class
    simulated_cabin_class = "Economy"

    # 3. Create our ReAct-based airline agent
    airline_agent = ReActAirlineAgent()

    # 4. Handle the refund request
    result_message = airline_agent.handle_refund_request(
        booking_time=simulated_booking_time,
        cabin_class=simulated_cabin_class
    )

    # 5. Print the final user-facing result
    print(result_message)

## How to Run
1. **Run all cells** in the notebook.
2. The output from the final cell should show you a message about whether the passenger is eligible for a refund.

You can adjust the `booking_time` and `cabin_class` in the **Main Orchestration** section to see different results.

## ReAct Pattern Recap
- **Reason**: The agent builds a chain-of-thought about what it needs to do.
- **Act**: The agent calls a policy-check function (the "tool") to gather data.
- **Observe**: The agent takes in the tool’s output.
- **Conclude**: The agent synthesizes a final, user-facing answer.

This approach makes the agent’s decisions **more interpretable and maintainable**, especially when integrated with more complex policy logic or multiple external tools.